#  run the lfnet

In [1]:
%env OMP_NUM_THREADS=10
%pylab inline
%load_ext autoreload
%autoreload 2

env: OMP_NUM_THREADS=10
Populating the interactive namespace from numpy and matplotlib


In [2]:
from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0" 
import sys
import numpy as np
import tensorflow as tf
import importlib
import time
import cv2
from tqdm import tqdm
import pickle

from datasets import *

from det_tools import *
from eval_tools import draw_keypoints
from common.tf_train_utils import get_optimizer
from imageio import imread, imsave
from inference import *
import matplotlib.pylab as plt
print('USE GPU #{} Host={}'.format(os.environ.get('CUDA_VISIBLE_DEVICES'), os.uname()[1]))

USE GPU #0 Host=multi-agent


In [3]:
def build_networks(config, photo,is_training):

    DET = importlib.import_module(config.detector)
    print(config.detector)
    detector = DET.Model(config,is_training)

    if config.input_inst_norm:
        print('Apply instance norm on input photos')
        photos1 = instance_normalization(photo)

    if config.use_nms3d:
        heatmaps, det_endpoints = build_multi_scale_deep_detector_3DNMS(config, detector, photo, reuse=False)
    else:
        heatmaps, det_endpoints = build_multi_scale_deep_detector(config, detector, photo, reuse=False)

    # extract patches
    kpts = det_endpoints['kpts']
    batch_inds = det_endpoints['batch_inds']

    kp_patches = build_patch_extraction(config, det_endpoints, photo)

    # Descriptor
    DESC = importlib.import_module(config.descriptor)
    descriptor = DESC.Model(config, is_training)
    desc_feats, desc_endpoints = build_deep_descriptor(config, descriptor, kp_patches, reuse=False) 
    # scale and orientation (extra)
    scale_maps = det_endpoints['scale_maps']
    ori_maps = det_endpoints['ori_maps'] # cos/sin
    degree_maps, _ = get_degree_maps(ori_maps) # degree (rgb psuedo color code)
    kpts_scale = det_endpoints['kpts_scale']
    kpts_ori = det_endpoints['kpts_ori']
    kpts_ori = tf.atan2(kpts_ori[:,1], kpts_ori[:,0]) # radian
    print(kpts_ori)
    
    ops = {
        'photo': photo,
        'is_training': is_training,
        'kpts': kpts,
        'feats': desc_feats,
        # EXTRA
        'scale_maps': scale_maps,
        'kpts_scale': kpts_scale,
        'degree_maps': degree_maps,
        'kpts_ori': kpts_ori,
        #"curr_in_zoe":desc_endpoints['curr_in_zoe'],
    }

    return ops

In [4]:
MODEL_PATH = './models'
if MODEL_PATH not in sys.path:
    sys.path.append(MODEL_PATH)
class arg:
    def __init__(self):
        self.num_threads = 8
        self.in_dir = '/home/zoe/data/rgbd_dataset_freiburg1_room/rgb/'
        self.out_dir = '/home/zoe/data/rgbd_dataset_freiburg1_room/LFNet_DBOW/'
        self.full_output = False
        self.model = './release/models/outdoor/'
        self.top_k = 1000
        self.max_longer_edge = 640
tmp_config = arg()

# restore other hyperparams to build model

In [5]:

if os.path.isdir(tmp_config.model):
    config_path = os.path.join(tmp_config.model, 'config.pkl')
else:
    config_path = os.path.join(os.path.dirname(tmp_config.model), 'config.pkl')
try:
    with open(config_path, 'rb') as f:
        config = pickle.load(f)
except:
    raise ValueError('Fail to open {}'.format(config_path))

for attr, dst_val in sorted(vars(tmp_config).items()):
    if hasattr(config, attr):
        src_val = getattr(config, attr)
        if src_val != dst_val:
            setattr(config, attr, dst_val)
    else:
        setattr(config, attr, dst_val)

# Build Networks

In [6]:
tf.reset_default_graph()

photo_ph = tf.placeholder(tf.float32, [1, None, None, 1]) # input grayscale image, normalized by 0~1
is_training = tf.constant(False) # Always False in testing

ops = build_networks(config, photo_ph, is_training)

tfconfig = tf.ConfigProto()
tfconfig.gpu_options.allow_growth = True 
sess = tf.Session(config=tfconfig)
sess.run(tf.global_variables_initializer())

mso_resnet_detector
Act-Fn:  <function get_activation_fn.<locals>.<lambda> at 0x7fe8c69f4620>
Apply instance norm on input photos
Tensor("MSDeepDet/ConvOnlyResNet/init_conv/BiasAdd:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-1/pre-bn/FusedBatchNorm:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-1/conv1/BiasAdd:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-1/mid-bn/FusedBatchNorm:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-1/add:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-2/pre-bn/FusedBatchNorm:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-2/conv1/BiasAdd:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-2/mid-bn/FusedBatchNorm:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor("MSDeepDet/ConvOnlyResNet/block-2/add:0", shape=(1, ?, ?, 16), dtype=float32)
Tensor(

# load model

In [7]:
saver = tf.train.Saver()
print('Load trained models...')

if os.path.isdir(config.model):
    checkpoint = tf.train.latest_checkpoint(config.model)
    model_dir = config.model
else:
    checkpoint = config.model
    model_dir = os.path.dirname(config.model)


if checkpoint is not None:
    print('Checkpoint', os.path.basename(checkpoint))
    print("[{}] Resuming...".format(time.asctime()))
    saver.restore(sess, checkpoint)
else:
    raise ValueError('Cannot load model from {}'.format(model_dir))    
print('Done.')

Load trained models...
Checkpoint models-latest-42000
[Fri Oct 26 17:09:46 2018] Resuming...
INFO:tensorflow:Restoring parameters from ./release/models/outdoor/models-latest-42000
Done.


# 保存pb文件

In [8]:
tf.train.write_graph(sess.graph_def, config.model, "lfnet_model_1000k.pbtxt", as_text=True)

'./release/models/outdoor/lfnet_model_1000k.pbtxt'

In [9]:
graph = sess.graph
print(sess.graph.get_operations()[-1])
print([node.name for node in graph.as_graph_def().node])

name: "save/restore_all"
op: "NoOp"
input: "^save/Assign"
input: "^save/Assign_1"
input: "^save/Assign_2"
input: "^save/Assign_3"
input: "^save/Assign_4"
input: "^save/Assign_5"
input: "^save/Assign_6"
input: "^save/Assign_7"
input: "^save/Assign_8"
input: "^save/Assign_9"
input: "^save/Assign_10"
input: "^save/Assign_11"
input: "^save/Assign_12"
input: "^save/Assign_13"
input: "^save/Assign_14"
input: "^save/Assign_15"
input: "^save/Assign_16"
input: "^save/Assign_17"
input: "^save/Assign_18"
input: "^save/Assign_19"
input: "^save/Assign_20"
input: "^save/Assign_21"
input: "^save/Assign_22"
input: "^save/Assign_23"
input: "^save/Assign_24"
input: "^save/Assign_25"
input: "^save/Assign_26"
input: "^save/Assign_27"
input: "^save/Assign_28"
input: "^save/Assign_29"
input: "^save/Assign_30"
input: "^save/Assign_31"
input: "^save/Assign_32"
input: "^save/Assign_33"
input: "^save/Assign_34"
input: "^save/Assign_35"
input: "^save/Assign_36"
input: "^save/Assign_37"
input: "^save/Assign_38"
i

# Ready to feed input images

In [10]:
img_paths = [x.path for x in os.scandir(config.in_dir) if x.name.endswith('.jpg') or x.name.endswith('.png')]
print('Found {} images...'.format(len(img_paths)))

if not os.path.exists(config.out_dir):
    os.makedirs(config.out_dir)

avg_elapsed_time = 0

for img_path in tqdm(img_paths):
    photo = imread(img_path)
    height, width = photo.shape[:2]
    longer_edge = max(height, width)
    if config.max_longer_edge > 0 and longer_edge > config.max_longer_edge:
        if height > width:
            new_height = config.max_longer_edge
            new_width = int(width * config.max_longer_edge / height)
        else:
            new_height = int(height * config.max_longer_edge / width)
            new_width = config.max_longer_edge
        photo = cv2.resize(photo, (new_width, new_height))
        height, width = photo.shape[:2]
    rgb = photo.copy()
    if photo.ndim == 3 and photo.shape[-1] == 3:
        photo = cv2.cvtColor(photo, cv2.COLOR_RGB2GRAY)
    photo = photo[None,...,None].astype(np.float32) / 255.0 # normalize 0-1
    assert photo.ndim == 4 # [1,H,W,1]
    
    feed_dict = {
        photo_ph: photo,
    }
    if config.full_output:
        fetch_dict = {
            'kpts': ops['kpts'],
            'feats': ops['feats'],
            'kpts_scale': ops['kpts_scale'],
            'kpts_ori': ops['kpts_ori'],
            'scale_maps': ops['scale_maps'],
            'degree_maps': ops['degree_maps'],
        }
        outs = sess.run(fetch_dict, feed_dict=feed_dict)
        # draw key
        kp_img = draw_keypoints(rgb, outs['kpts'])
        scale_range = config.net_max_scale-config.net_min_scale
        if scale_range == 0:
            scale_range = 1.0
        scale_img = (outs['scale_maps'][0]*255/scale_range).astype(np.uint8)
        ori_img = (outs['degree_maps'][0]*255).astype(np.uint8)
        out_img_path = os.path.join(config.out_dir, os.path.basename(img_path))
        imsave(out_img_path, kp_img)
        imsave(out_img_path+'-scl.jpg', scale_img)
        imsave(out_img_path+'-ori.jpg', ori_img)
        np.savez(out_img_path+'.npz', kpts=outs['kpts'], descs=outs['feats'], size=np.array([height, width]),
                 scales=outs['kpts_scale'], oris=outs['kpts_ori'])
    else:
        # Dump keypoint locations and their features
        fetch_dict = {
            'kpts': ops['kpts'],
            'feats': ops['feats'],
            'kpts_ori': ops['kpts_ori'],
            'kpts_scale': ops['kpts_scale'],
            #'curr_in_zoe': ops['curr_in_zoe'],
        }
        outs = sess.run(fetch_dict, feed_dict=feed_dict)
        out_kpts_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_kpts.txt')
        out_feats_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_feats.txt')
        out_kpts_ori_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_kpts_ori.txt')
        out_kpts_scale_path = os.path.join(config.out_dir, os.path.splitext(os.path.basename(img_path))[0]+'_kpts_scale.txt')
        #print(outs['curr_in_zoe'])
        numpy.savetxt(out_kpts_path, outs['kpts'])
        numpy.savetxt(out_feats_path, outs['feats'])
        numpy.savetxt(out_kpts_ori_path, outs['kpts_ori']*180/np.pi+180)
        numpy.savetxt(out_kpts_scale_path, outs['kpts_scale'])
        #np.savez(out_path, kpts=outs['kpts'], feats=outs['feats'], size=np.array([height, width]))
print('Done.')

  0%|          | 0/1362 [00:00<?, ?it/s]

Found 1362 images...


100%|██████████| 1362/1362 [07:23<00:00,  3.15it/s]

Done.


## View keypoints from dumped npz

In [11]:
img_name = '00204549_3820006713.jpg'
img_path = '/home/zoe/deeplearning/lfnet/dump_feats/'+img_name
kp_path = '/home/zoe/deeplearning/lfnet/dump_feats/{}.npz'.format(img_name)
image = imread(img_path).astype(np.float32) / 255
kp_info = np.load(kp_path)
kpts = kp_info['kpts']
feats = kp_info['descs']
kpts_ori = kp_info['oris']
kpts_scales = kp_info['scales']
height, width = kp_info['size']
image = cv2.resize(image, (width, height))
print('Load {}-kpts, feat={}-D imsize={}x{}'.format(kpts.shape[0], feats.shape[1], width, height))

Load 500-kpts, feat=256-D imsize=640x420


In [12]:
[1.41421356 1.18920712 1.         0.84089642 0.70710678]

SyntaxError: invalid syntax (<ipython-input-12-354d5bb7b37d>, line 1)

In [ ]:
print(kpts_scales)

In [ ]:
plt.imshow(image)
plt.plot(
    kpts[:,0], kpts[:,1],
    alpha=1,
    linestyle='none',
    linewidth=0,
    aa=False,
    marker='.',
    markersize=2,
    color=[0.9, 0.1, 0.1],
);